# Body Language Decoder

## Table of Content
0. [Import Dependencies](#install)
1. [Detection using MediaPipe](#detection) 
2. [Feature Extraction](#feature-extraction) 
    1. [Write Columns Head in CSV File](#csv-header)
    2. [Extract Features of Assigned Class](#save-coordinates)
3. [Train Custom Model Using Scikit Learn](#model)
    1. [Load and Preprocess Input Data](#load-input)
    2. [Train Machine Learning Classification Models](#training)
    3. [Evaluate and Serialize Model](#evaluate)
4. [Real-time Detections with Model](#real-time-detection)

## 0. Import Dependencies <a id="install"></a>

In [11]:
from mediapipe import solutions as mp
import cv2
import csv
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
import pickle
import os

## 1. Detection using MediaPipe <a id="detection"></a>

In [3]:
#Check Parameter that can put
mp.holistic.Holistic??

In [2]:
cap = cv2.VideoCapture(0)

# Initiate holistic model
with mp.holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:

    while cap.isOpened():
        success, frame = cap.read()

        if not success:
            break

        # Recolor feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Make detections
        results = holistic.process(image)

        # Recolor for rendering
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Draw face landmarks
        mp.drawing_utils.draw_landmarks(image, results.face_landmarks, mp.holistic.FACE_CONNECTIONS,
                                        mp.drawing_utils.DrawingSpec(
                                            color=(242, 216, 175), thickness=1, circle_radius=1),
                                        mp.drawing_utils.DrawingSpec(color=(242, 216, 175), thickness=1))

        # Right hand
        mp.drawing_utils.draw_landmarks(image, results.right_hand_landmarks, mp.holistic.HAND_CONNECTIONS,
                                        mp.drawing_utils.DrawingSpec(
                                            color=(181, 137, 82), thickness=2, circle_radius=4),
                                        mp.drawing_utils.DrawingSpec(color=(181, 137, 82), thickness=2))

        # Left hand
        mp.drawing_utils.draw_landmarks(image, results.left_hand_landmarks, mp.holistic.HAND_CONNECTIONS,
                                        mp.drawing_utils.DrawingSpec(
                                            color=(181, 137, 82), thickness=2, circle_radius=4),
                                        mp.drawing_utils.DrawingSpec(color=(181, 137, 82), thickness=2))

        # Pose 
        mp.drawing_utils.draw_landmarks(image, results.pose_landmarks, mp.holistic.POSE_CONNECTIONS,
                                        mp.drawing_utils.DrawingSpec(
                                            color=(65, 53, 31), thickness=2, circle_radius=2),
                                        mp.drawing_utils.DrawingSpec(color=(65, 53, 31), thickness=2))

        cv2.imshow("Holistic Model Detection", image)
        
        # Press "q" to exit
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

In [4]:
results.pose_landmarks

landmark {
  x: 0.35972744
  y: 0.5396109
  z: -1.4723308
  visibility: 0.9996866
}
landmark {
  x: 0.39126045
  y: 0.47113445
  z: -1.4457468
  visibility: 0.9993681
}
landmark {
  x: 0.41233438
  y: 0.471114
  z: -1.4454756
  visibility: 0.99947083
}
landmark {
  x: 0.431556
  y: 0.47210085
  z: -1.4463551
  visibility: 0.99940836
}
landmark {
  x: 0.3246643
  y: 0.47421175
  z: -1.4445863
  visibility: 0.99921805
}
landmark {
  x: 0.30272996
  y: 0.4758924
  z: -1.445382
  visibility: 0.9992656
}
landmark {
  x: 0.28038245
  y: 0.4772425
  z: -1.4461673
  visibility: 0.99907553
}
landmark {
  x: 0.4664002
  y: 0.5028918
  z: -1.0384543
  visibility: 0.9994807
}
landmark {
  x: 0.2515541
  y: 0.50804985
  z: -1.0548478
  visibility: 0.99900836
}
landmark {
  x: 0.401982
  y: 0.6252474
  z: -1.2978094
  visibility: 0.9996053
}
landmark {
  x: 0.31848273
  y: 0.6259609
  z: -1.2966603
  visibility: 0.9994773
}
landmark {
  x: 0.59410924
  y: 0.8014133
  z: -0.69179815
  visibility: 0.9

## 2. Feature Extraction <a id="feature-extraction"></a>

### Write Columns Head in CSV File <a id="csv-header"></a>

In [12]:
num_coords = len(results.pose_landmarks.landmark) + len(results.face_landmarks.landmark)
num_coords

501

In [13]:
landmarks = ['class']
for val in range(1, num_coords+1):
    landmarks += ['x{}'.format(val), 'y{}'.format(val),
                  'z{}'.format(val), 'v{}'.format(val)]

In [14]:
if not os.path.exists("data"):
    os.mkdir("data")

with open("data/body_language_coords.csv", mode="w", newline="" ) as f:
    csv_writer = csv.writer(f, delimiter=",", quotechar='"',quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

### Extract Features of Assigned Class <a id="save-coordinates"></a>

- Repeat this step to save features of different targets.
- Make sure your samples cover different scenario of the target.

In [11]:
class_name = "right hand"

In [12]:
cap = cv2.VideoCapture(0)
with mp.holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:

    while cap.isOpened():
        success, frame = cap.read()

        if not success:
            break
            
        # Recolor feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Make detections
        results = holistic.process(image)

        # Recolor for rendering
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Draw face landmarks
        mp.drawing_utils.draw_landmarks(image, results.face_landmarks, mp.holistic.FACE_CONNECTIONS,
                                        mp.drawing_utils.DrawingSpec(
                                            color=(242, 216, 175), thickness=1, circle_radius=1),
                                        mp.drawing_utils.DrawingSpec(color=(242, 216, 175), thickness=1))

        # Right hand
        mp.drawing_utils.draw_landmarks(image, results.right_hand_landmarks, mp.holistic.HAND_CONNECTIONS,
                                        mp.drawing_utils.DrawingSpec(
                                            color=(181, 137, 82), thickness=2, circle_radius=4),
                                        mp.drawing_utils.DrawingSpec(color=(181, 137, 82), thickness=2))

        # Left hand
        mp.drawing_utils.draw_landmarks(image, results.left_hand_landmarks, mp.holistic.HAND_CONNECTIONS,
                                        mp.drawing_utils.DrawingSpec(
                                            color=(181, 137, 82), thickness=2, circle_radius=4),
                                        mp.drawing_utils.DrawingSpec(color=(181, 137, 82), thickness=2))

        # Pose 
        mp.drawing_utils.draw_landmarks(image, results.pose_landmarks, mp.holistic.POSE_CONNECTIONS,
                                        mp.drawing_utils.DrawingSpec(
                                            color=(65, 53, 31), thickness=2, circle_radius=2),
                                        mp.drawing_utils.DrawingSpec(color=(65, 53, 31), thickness=2))
        
        # Export coordinates
        if results.pose_landmarks and results.face_landmarks:
            # Extract pose landmarks
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] 
                                      for landmark in pose]).flatten())

                
            # Extract face landmarks
            face = results.face_landmarks.landmark
            face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] 
                                      for landmark in face]).flatten())
            
#             # Extract right hand landmarks
#             right_hand = results.right_hand_landmarks.landmark
#             right_hand_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] 
#                                             for landmark in right_hand]).flatten())
           
            # Concate row
            row = pose_row + face_row 
            
            # Append class name
            row.insert(0, class_name)
            
            # Export to CVS
            with open("data/body_language_coords.csv", mode="a", newline="" ) as f:
                csv_writer = csv.writer(f, delimiter=",", quotechar='"',quoting=csv.QUOTE_MINIMAL)
                csv_writer.writerow(row)
        
        cv2.imshow("Holistic Model Detection", image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

## 3. Train Custom Model Using Scikit Learn <a id="model"></a>

### Load and Preprocess Input Data <a id="load-input"></a>

In [15]:
df = pd.read_csv("data/body_language_coords.csv")
df

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501


In [4]:
X = df.drop("class", axis=1)
y = df["class"]

In [15]:
y.value_counts()

right hand    274
x cross       170
left hand     147
Name: class, dtype: int64

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=47)

### Train Machine Learning Classification Models <a id="training"></a>

In [9]:
pipelines = {
    "lr":make_pipeline(StandardScaler(), LogisticRegression()),
    "svc":make_pipeline(StandardScaler(), SVC()),
    "rf":make_pipeline(StandardScaler(), RandomForestClassifier()),
    "knn":make_pipeline(StandardScaler(), KNeighborsClassifier()),
}

In [10]:
fit_models = {}
for algo, pipeline in pipelines.items():
    model = pipeline.fit(X_train, y_train.ravel())
    fit_models[algo] = model

AttributeError: 'str' object has no attribute 'decode'

### Evaluate and Serialize Model <a id="evaluate"></a>

In [8]:
# Show accuracy
for algo, model in fit_models.items():
    pred = model.predict(X_test)
    print(algo, accuracy_score(y_test, pred))

In [19]:
# Show confusion matrix
for algo, model in fit_models.items():
    pred = model.predict(X_test)
    print(algo, confusion_matrix(y_test, pred), sep="\n", end="\n\n")

In [36]:
# Create folder if it does not exist
if not os.path.exists("generated_model"):
    os.mkdir("generated_model")
    
# Save model to file
model_to_save = "rf"

with open("generated_model/body_language_model.pkl", "wb") as f:
    pickle.dump(fit_models[model_to_save], f)

## 4. Real-time Detections with Model <a id="real-time-detection"></a>

In [37]:
with open("generated_model/body_language_model.pkl", "rb") as f:
    model_inference = pickle.load(f)

In [ ]:
cap = cv2.VideoCapture(0)
with mp.holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:

    while cap.isOpened():
        success, frame = cap.read()

        if not success:
            break

        # Recolor feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Make detections
        results = holistic.process(image)


        # Recolor for rendering
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Draw face landmarks
        mp.drawing_utils.draw_landmarks(image, results.face_landmarks, mp.holistic.FACE_CONNECTIONS,
                                        mp.drawing_utils.DrawingSpec(
                                            color=(242, 216, 175), thickness=1, circle_radius=1),
                                        mp.drawing_utils.DrawingSpec(color=(242, 216, 175), thickness=1))

        # Right hand
        mp.drawing_utils.draw_landmarks(image, results.right_hand_landmarks, mp.holistic.HAND_CONNECTIONS,
                                        mp.drawing_utils.DrawingSpec(
                                            color=(181, 137, 82), thickness=2, circle_radius=4),
                                        mp.drawing_utils.DrawingSpec(color=(181, 137, 82), thickness=2))

        # Left hand
        mp.drawing_utils.draw_landmarks(image, results.left_hand_landmarks, mp.holistic.HAND_CONNECTIONS,
                                        mp.drawing_utils.DrawingSpec(
                                            color=(181, 137, 82), thickness=2, circle_radius=4),
                                        mp.drawing_utils.DrawingSpec(color=(181, 137, 82), thickness=2))

        # Pose 
        mp.drawing_utils.draw_landmarks(image, results.pose_landmarks, mp.holistic.POSE_CONNECTIONS,
                                        mp.drawing_utils.DrawingSpec(
                                            color=(65, 53, 31), thickness=2, circle_radius=2),
                                        mp.drawing_utils.DrawingSpec(color=(65, 53, 31), thickness=2))
        
        # Export coordinates
        if results.pose_landmarks and results.face_landmarks:
            # Extract pose landmarks
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] 
                                      for landmark in pose]).flatten())

                
            # Extract face landmarks
            face = results.face_landmarks.landmark
            face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] 
                                      for landmark in face]).flatten())
           
            # Concate row
            row = pose_row + face_row 
            
            # Predict using inference model
            X = pd.DataFrame([row])
            pred = model_inference.predict(X)[0]
            prob = np.max(model_inference.predict_proba(X)[0]).round(2)
            print(pred, prob)
            
            # Display result
            cv2.rectangle(image, (0,0), (250,60), (245, 117, 16), -1)
            cv2.putText(image, "CLASS", (95,12), cv2.FONT_HERSHEY_SIMPLEX,
                       0.5, (0,0,0), 1, cv2.LINE_AA)
            cv2.putText(image, pred, (90,40), cv2.FONT_HERSHEY_SIMPLEX,
                       1, (255, 255, 255), 2, cv2.LINE_AA)
            cv2.putText(image, "PROB", (15,12), cv2.FONT_HERSHEY_SIMPLEX,
                       0.5, (0,0,0), 1, cv2.LINE_AA)
            cv2.putText(image, str(prob), (10,40), cv2.FONT_HERSHEY_SIMPLEX,
                       1, (255, 255, 255), 2, cv2.LINE_AA)
        
        
        cv2.imshow("Holistic Model Detection", image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()